# Sentimental analysis

## Classical classification - Finbert Sentiment

In [1]:
# finbert sentiment
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="ProsusAI/finbert")

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

c:\Users\woojin\anaconda3\envs\llm\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\woojin\.cache\huggingface\hub\models--ProsusAI--finbert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [2]:
print(pipe("Stock rallied yesterday"))

[{'label': 'positive', 'score': 0.7569195628166199}]


In [4]:
print(pipe("Tesla stock dropped by 25%, but Meta rallied 30%"))
print(pipe("Meta rallied 30% but Tesla stock dropped by 25%"))

[{'label': 'negative', 'score': 0.9681780338287354}]
[{'label': 'negative', 'score': 0.9746080040931702}]


In [8]:
print(pipe("Meta reported strong financial result for Q1 2024"))
print(pipe("Tesla released its financial results for Q1 2024, missing Wall Street expectations."))

[{'label': 'positive', 'score': 0.9483426809310913}]
[{'label': 'negative', 'score': 0.9690130949020386}]


# Generative sentimental analysis

In [9]:
from langchain_community.chat_models import ChatOllama
from openai import OpenAI
import keyring

llm_llama = ChatOllama(model='llama3.1')
llm_openai = OpenAI(
    api_key=keyring.get_password('openai', 'key_for_windows')
)

def get_response_from_llama(prompt):
    response = llm_llama.invoke(prompt)
    return response.content

def get_response_from_openai(prompt):
    completion = llm_openai.chat.completions.create(
        messages=[
            {'role':'user', 'content':prompt}
        ],
        model='gpt-3.5-turbo',
        max_tokens=1024,
        temperature=0.0
    )
    return completion.choices[0].message.content

In [10]:
text_base = "삼성전자는 연결 기준 올해 1분기 영업이익이 6조 6천억으로 지난해 동기보다 931.25% 증가한 것으로 잠정 집계됐다고 5일 공시했다."

prompt = f"""아래 문장을 긍정, 부정, 중립 중 하나로 감정 분석해줘.
# 문장
{text_base}"""
answer = get_response_from_llama(prompt)
print(answer)

긍정적인 문장입니다. (영업이익이 증가했음)


In [12]:
prompt = f'''아래 문장을 긍정, 부정, 중립 중 하나로 감정 분석해줘.
긍정, 부정, 중립 중 분류한 다음, 결과만 대답해줘.

아래 포맷 형식대로 작성해줘.
{{"감성":<긍정/부정/중립>, "원문":<텍스트>}}
# 문장
{text_base}'''

answer = get_response_from_llama(prompt)
print(answer)

{"감성":긍정, "원문":"삼성전자는 연결 기준 올해 1분기 영업이익이 6조 6천억으로 지난해 동기보다 931.25% 증가한 것으로 잠정 집계됐다고 5일 공시했다."}


In [13]:
text_base_revised = """삼성전자는 연결 기준 올해 1분기 영업이익이 6조 6천억으로 지난해 동기보다 931.25% 증가한 것으로 잠정 집계됐다고 5일 공시했다.
반면에, 순양전자는 올해 1분기 영업이익이 3천억으로 지난해보다 23% 감소한 것으로 공시했습니다."""

prompt = f'''아래 문장을 긍정, 부정, 중립 중 하나로 감정 분석해줘.
긍정, 부정, 중립 중 분류한 다음, 결과만 대답해줘.

아래 포맷 형식대로 작성해줘.
{{"감성":<긍정/부정/중립>, "원문":<텍스트>}}
# 문장
{text_base_revised}'''

answer = get_response_from_llama(prompt)
print(answer)


{"감성":<긍정>, "원문":"삼성전자는 연결 기준 올해 1분기 영업이익이 6조 6천억으로 지난해 동기보다 931.25% 증가한 것으로 잠정 집계됐다고 5일 공시했다. 반면에, 순양전자는 올해 1분기 영업이익이 3천억으로 지난해보다 23% 감소한 것으로 공시했습니다."}


In [15]:
prompt = f'''아래 문장을 긍정, 부정, 중립 중 하나로 감정 분석해줘.
긍정, 부정, 중립 중 분류한 다음, 결과만 대답해줘.

아래 포맷 형식대로 작성해줘.
{{{{"감성":<긍정/부정/중립>, "원문":<텍스트1>}}, {{"감성":<긍정/부정/중립>, "원문":<텍스트2>}}}}
# 문장
{text_base_revised}'''

answer = get_response_from_llama(prompt)
print(answer)


{"감성":중립, "원문":삼성전자는 연결 기준 올해 1분기 영업이익이 6조 6천억으로 지난해 동기보다 931.25% 증가한 것으로 잠정 집계됐다고 5일 공시했다.}
{"감성":부정, "원문":반면에, 순양전자는 올해 1분기 영업이익이 3천억으로 지난해보다 23% 감소한 것으로 공시했습니다.}
